## **Preface**
This goal of this guide to introduce USD concepts without any code, where we read and write raw `.usda` files. also, we learn about why Composition arcs can lead to confusion.

> Regards USD Terminology, we'll pick what we need, when we need it.


## **Introduction**
Each USD file ( `.usda`, `.usdc`, `.usd`, `.usdz` ) represents a USD **Layer**.

Each USD Layer can consist of one or more **Prims** or USD **Layers**.

Each **Prim** (USD Object) has:

-  A **Schema** (Type)
-  A **Namespace** (Path in the hierarchy)
-  **Properties**
    -   **Attributes** (e.g. Points, Normal, UVs, ... ,and etc. )
    -   **Relationships** (e.g. Material Binding)
-   **Metadata** (Static Info.)

  
> I find it funny that we need to understand the file structure of a USD file unlike we don't do that with alembics and FBXs.  
> So, we may need to open USD files as text files but we don't do that with FBX files for example.

## **Attributes**

### **Attribute Template**

Each **Schema** has an **attribute** template.

e.g. few examples.

> I'm using Houdini for visualization  
> if you are wondering about attributes color code in Houdini. it's mentioned [here](https://www.sidefx.com/docs/houdini/ref/panes/scenegraphdetails.html#text-colors)

| **Xfrom** | **mesh** | **camera** |
| ---| ---| --- |
| ![](../resources/getting_started_guide/xform.png) | ![](../resources/getting_started_guide/mesh.png) | ![](../resources/getting_started_guide/camera.png) |

### **Attribute Data Type**

Each Attribute has a data type

*   integer
*   float
*   string
*   vector
*   array

  

### **PrimVars**

**PrimVars** attributes: any custom attributes found on the geometry will be added to USD as `PrimVars:my_attribute`

Each **PrimVar** has an **interpolation** mode depends on the attribute class.

Attribute Class Interpolation

Point ➝ Vertex

Vertex ➝ Face Varying

Primitive ➝ Uniform

Detail ➝ Constant

  

### **Default vs. Time Samples (Static vs. Animated)**

  

two static points

```python
points[2]:[
            (1,0,0),
            (1,0.5,0),
          ]
```

two animated points

```python
points[2]:{
           1:[(1,0,0),(1,0.5,0)],
           2:[(0.9,0,0),(1,0.4,0)],
           3:[(0.8,0,0),(1,0.3,0)],
          }
```

Having animated attributes can multiply your file size. so, make sure that static attributes don't have time samples.

  

### **Time Samples**

USD uses linear interpolation for frames in between time samples.

  

```python
primvars:test[2]:{
                   1:[0],
                   2:[5]
                  }
```

  

At **frame\[1.5\]** value ➝ (0+5)/2 = **2.5**

  

### **Fallbacks**

Attribute templates provided by schemas come with default fall back values if users didn't define any values

e.g. if you didn't set orientation, USD will assume it's rightHanded.

![fallbacks](../resources/getting_started_guide/fallbacks.png)


## **Layers**

### **Different Formats**

In essence. Each USD file ( `.usda`, `.usdc`, `.usd`, `.usdz` ) represents a USD **Layer**.

*   `.usda` : USD Ascii, Human-readable (used for debugging).
*   `.usdc` : USD Crate. Binary (a compressed USD file, memory efficient and fast accessed).
*   `.usd` : Can be and Ascii or Crate.
*   `.usdz` : USD ZIP, A USD layer and its supporting files like textures.

  

USD comes with tool `usdcat` that converts between different USD formats

### **Layer Example**


In [1]:
%%file examples/usd_beginner_guide/layer_example.usda
#usda 1.0
# sphere.usda
(
    # Optional metadata for the layer
    framesPerSecond = 24
    upAxis = "Y"
    doc = "This is sample ascii usd file"
)

def Sphere "my_sphere"(
    # Optional metadata for the prim
    kind = "component"
    doc = "This is sphere with an animated attribute."
)
{
    matrix4d xformOp:transform = ((1, 0, 0, 0), 
                                  (0, 1, 0, 0), 
                                  (0, 0, 1, 0), 
                                  (0, 0, 0, 1))
    uniform token[] xformOpOrder = ["xformOp:transform"]

    float primvars:test.timeSamples = {
            1: 0.84147096,
            2: 0.9092974,
            3: 0.14112,
            4: -0.7568025,
            5: -0.9589243,
            6: -0.2794155,
            7: 0.6569866,
            8: 0.98935825,
            9: 0.4121185,
            10: -0.5440211,
        }
}

Writing examples/usd_beginner_guide/layer_example.usda


## **Composition**

Composition refers to how layers are composed/combined

For example, Each department can have its own layer where it adds  its contribution to the the final shot layer.

Each department can author its opinions on different layers by adding prims or modifying existing ones.

Imagine a list of departments and we have ranked/ordered that list from weakest to strongest.

e.g. `Model` ➜ `Layout` ➜ `Animation` ➜ `Simulation` ➜ `FX` ➜ `Lighting`

The higher the rank/order, the stronger the opinion.

So, when many departments have different opinions about the position of an object and the opinion of the strongest department wins.

Responsible for merging these layers and views (based on your input) is the **compositing engine**.

### **Stage**

A Stage is the resulting/composed scene graph generated by the composition engine.

### **Composition Arcs**

Composition **Arcs are** Composition **Operators.**

We use arcs/operators to define how should the departments layers be combined..

Each arc/operator adds a certain strength to a layer.

It's called **arc** because when you draw a diagram to show how they are combined, you draw an arc that goes from one layer to the other.

There are many available Arc/Operators as follows:

#### **Sub-Layering**

Sub-Layering creates a **layer stack** where layers are stacked on top of each other from lowest to highest order.

The higher the order, the stronger the opinion.

Here's a demo where I sub-layered layers from different departments.


In [14]:
%%html
<video width=800 controls>
  <source src="../resources/getting_started_guide/sublayer_example.mp4" type="video/mp4">
</video> 

In the previous example: file `cube_asset.usda` is exactly as:

In [81]:
from pxr import Usd
# This USDA file was created in Houdini.
stage = Usd.Stage.Open('examples/usd_beginner_guide/sublayer_example/cube_asset.usda')
print(stage.GetRootLayer().ExportToString())

#usda 1.0
(
    endTimeCode = 90
    framesPerSecond = 24
    metersPerUnit = 1
    startTimeCode = 1
    subLayers = [
        @./animation.usda@,
        @./lookdev.usda@,
        @./model.usda@
    ]
    timeCodesPerSecond = 24
    upAxis = "Y"
)




The layers order from weak to strong is from top to bottom:

1. cube_asset.usda (itself)
2. animation.usda
3. lookdev.usda
4. model.usda

#### **Reference**
It's a pointer to a specific Prim on a layer.

It's suitable to build assets.

you can reference the same layer more than once. (unlike sub-layering)
> I've included the necessary files from `esper_room` asset in the example files so that the following layer will work properly.
> 
> I don't own that asset. Find the full asset here [3D Assets | JCube](https://j-cube.jp/solutions/multiverse/assets/).
> Many thanks to JCube for making this asset available.

In [8]:
%%file examples/usd_beginner_guide/reference_example.usda
#usda 1.0

def Xform "Bedroom"
{
  def "Cover"(
    prepend references  = @./esper_room_v3/bed/bed_geo.usdc@</Bed/BedCover> 
  )
  {}

  def "Drawer01"(
    prepend references  = @./esper_room_v3/clothDrawer/clothesDrawer_geo.usdc@</ClothesDrawer> 
  )
  {
    double3 xformOp:translate = (-6.0, 0.0, -12.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
  }

  def "Drawer02"(
    prepend references  = @./esper_room_v3/clothDrawer/clothesDrawer_geo.usdc@</ClothesDrawer> 
  )
  {
    double3 xformOp:translate = (6.0, 0.0, -12.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
  }
}

Writing examples/usd_beginner_guide/reference_example.usda


![reference example](../resources/getting_started_guide/reference_example.png)



#### **Specifier**

specifies how the Prim should be interpreted during composition, there are 3 specifiers:

*   `def` ➝ define
*   `over` ➝ override (weakest)
*   `class` ➝ class

In [11]:
%%file examples/usd_beginner_guide/override_example.usda
#usda 1.0
  
# the data in this layer is meaningles, if you don't look at it in the right context.
# and this supposed to be looked togther with the model.
(
    subLayers = [
        @./esper_room_v3/clothDrawer/clothesDrawer_geo.usdc@
    ]
)

over "ClothesDrawer"
{
    # I'm moving the whole ClothesDrawer. but, it's not shown in my screen shots.
    double3 xformOp:translate = (6, 0, -12)
    uniform token[] xformOpOrder = ["xformOp:translate"]

    over "Drawer"
    {
        over "DrawerMid"
        {
            double3 xformOp:translate = (0, 0, 1.5)
            uniform token[] xformOpOrder = ["xformOp:translate"]
        }

        over "DrawerTop"
        {
            double3 xformOp:translate = (0, 0, 0.25)
            uniform token[] xformOpOrder = ["xformOp:translate"]
        }
    }
}

Writing examples/usd_beginner_guide/override_example.usda


| Original | Override |
| -- | -- | 
| ![](../resources/getting_started_guide/before_override.png) | ![](../resources/getting_started_guide/after_override.png) |



### **Composition Arcs - In depth**

Each composition arc works in a different level/context.

*   Variant ➝ Prim Context
*   Sub-Layer ➝ Layer Context
*   Reference ➝ Prim Context
*   Payload ➝ Prim Context
*   Inherit ➝ Prim Context
*   Specialize ➝ Prim Context

#### **Variants**
Well, variants are used to add ehm.. variants 🤷‍♂️.

**Example : Add model variation**

In [12]:
%%file examples/usd_beginner_guide/variant_example.usda
#usda 1.0

def Xform "Xform" (
    variants = {
        string model = "Default_Cube"
    }
    prepend variantSets = "model"
)
{
    variantSet "model" = {
        "Default_Cube" {
            def Cube "Default_Cube"
            {
                matrix4d xformOp:transform = ( (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 1, 0, 1) )
                uniform token[] xformOpOrder = ["xformOp:transform"]
            }
        }
        "Red_Tall_Cube" {
            def Cube "Red_Tall_Cube"
            {
                color3f[] primvars:displayColor = [(1, 0, 0)] (
                    interpolation = "constant"
                )
                matrix4d xformOp:transform = ( (1, 0, 0, 0), (0, 2, 0, 0), (0, 0, 1, 0), (0, 2, 0, 1) )
                uniform token[] xformOpOrder = ["xformOp:transform"]
            }
        }
        "Blue_Short_Cube" {
            def Cube "Blue_Short_Cube"
            {
                color3f[] primvars:displayColor = [(0, 0, 1)] (
                    interpolation = "constant"
                )
                matrix4d xformOp:transform = ( (1, 0, 0, 0), (0, 0.5, 0, 0), (0, 0, 1, 0), (0, 0.5, 0, 1) )
                uniform token[] xformOpOrder = ["xformOp:transform"]
            }
        }
    }
}

Writing examples/usd_beginner_guide/variant_example.usda


**Result:**

![](../resources/getting_started_guide/variant_example.gif)

#### **Sub-Layer**

**Sub-Layer** is used to add/stack different layers in one file

**Example 1 : Sub-layers with different Prims.**

In [23]:
%%file examples/usd_beginner_guide/sublayer_example_1_different_prims/a_sphere.usda
#usda 1.0

def Sphere "a_sphere"
{}

Overwriting examples/usd_beginner_guide/sublayer_example_1_different_prims/a_sphere.usda


In [24]:
%%file examples/usd_beginner_guide/sublayer_example_1_different_prims/a_cube.usda
#usda 1.0

def Cube "a_cube"
{
    double3 xformOp:translate = (0.0, 0.0, 2.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
}

Overwriting examples/usd_beginner_guide/sublayer_example_1_different_prims/a_cube.usda


In [25]:
%%file examples/usd_beginner_guide/sublayer_example_1_different_prims/sublayer_cube_sphere.usda
#usda 1.0

# you can't sublayer the same layer twice (I can't add 'a_cube.usda' twice!).
# These layers order don't matter as they won't affect each other as both have different Prims!
(
    subLayers = [
        @./a_cube.usda@,
        @./a_sphere.usda@
    ]
)

Overwriting examples/usd_beginner_guide/sublayer_example_1_different_prims/sublayer_cube_sphere.usda


**Result:**
![](../resources/getting_started_guide/sublayer_example_1_different_prims.png)

**Example 2 : Sub-layers with the same Prim.**

Note: I'm using the same prim name. So, both layers have an opinion on the prim display color. which opinion will win?

In [26]:
%%file examples/usd_beginner_guide/sublayer_example_2_same_prim/a_red_cube.usda
#usda 1.0

def Cube "a_cube"
{
    color3f[] primvars:displayColor = [(1, 0, 0)] (
        interpolation = "constant"
    )
}

Writing examples/usd_beginner_guide/sublayer_example_2_same_prim/a_red_cube.usda


In [27]:
%%file examples/usd_beginner_guide/sublayer_example_2_same_prim/a_blue_cube.usda
#usda 1.0

def Cube "a_cube"
{
    color3f[] primvars:displayColor = [(0, 0, 1)] (
        interpolation = "constant"
    )
}

Writing examples/usd_beginner_guide/sublayer_example_2_same_prim/a_blue_cube.usda


In [28]:
%%file examples/usd_beginner_guide/sublayer_example_2_same_prim/sublayer_two_cubes.usda
#usda 1.0

# These cubes have the same prim. 
# So, what color will be the cube ? 
(
    subLayers = [
        @./a_blue_cube.usda@,
        @./a_red_cube.usda@
        
    ]
)

Writing examples/usd_beginner_guide/sublayer_example_2_same_prim/sublayer_two_cubes.usda


**Result:** Notice the layer stack.
![](../resources/getting_started_guide/sublayer_example_2_same_prims.png)


**Example 3 : Sub-layers with the same Prim. Author opinions as sub-layers.**

In [30]:
%%file examples/usd_beginner_guide/sublayer_example_3_same_prim/a_blue_cube.usda
#usda 1.0

def Cube "a_cube"
{
    color3f[] primvars:displayColor = [(0, 0, 1)] (
        interpolation = "constant"
    )
}

Writing examples/usd_beginner_guide/sublayer_example_3_same_prim/a_blue_cube.usda


In [31]:
%%file examples/usd_beginner_guide/sublayer_example_3_same_prim/turn_red.usda
#usda 1.0

# This layer has no meaning on its own, it's supposed to be sublayered.
over "a_cube"
{
    color3f[] primvars:displayColor = [(1, 0, 0)] (
        interpolation = "constant"
    )
}

Writing examples/usd_beginner_guide/sublayer_example_3_same_prim/turn_red.usda


In [32]:
%%file examples/usd_beginner_guide/sublayer_example_3_same_prim/sublayer_two_cubes.usda
#usda 1.0

# we can use sub-layers to author opnions / alter properties.
(
    subLayers = [
        @./turn_red.usda@,
        @./a_blue_cube.usda@,
    ]
)

Writing examples/usd_beginner_guide/sublayer_example_3_same_prim/sublayer_two_cubes.usda


**Result**: Notice the layer stack.
![](../resources/getting_started_guide/sublayer_example_3_same_prims.png)

#### **Reference**
References are used to refer to other prims either in the same layer or in another layer.

**Example 1: Reference a layer**

In [34]:
%%file examples/usd_beginner_guide/reference_example_1_reference_a_layer/a_sphere.usda
#usda 1.0

def Sphere "a_sphere"
{}

Writing examples/usd_beginner_guide/reference_example_1_reference_a_layer/a_sphere.usda


In [35]:
%%file examples/usd_beginner_guide/reference_example_1_reference_a_layer/reference_a_sphere.usda
#usda 1.0

# referencing is done by choosing your layer followed by
# the desired prim to be referenced.
def "sphere_01"(
    prepend references = @./a_sphere.usda@</a_sphere> 
)
{} 
# you can reference the same Prim as much as you want.
def "sphere_02"(
    prepend references = @./a_sphere.usda@</a_sphere>
)
{
    double3 xformOp:translate = (0.0, 0.0, 2.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 
def "sphere_03"(
    prepend references = @./a_sphere.usda@</a_sphere>
)
{
    double3 xformOp:translate = (0.0, 0.0, 4.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 
def "sphere_04"(
    prepend references = @./a_sphere.usda@</a_sphere>
)
{
    double3 xformOp:translate = (0.0, 0.0, 6.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 

Writing examples/usd_beginner_guide/reference_example_1_reference_a_layer/reference_a_sphere.usda


**Result**:
![](../resources/getting_started_guide/reference_example_1_reference_a_layer.png)


**Example 2: Reference and Sublayer**

In [36]:
%%file examples/usd_beginner_guide/reference_example_2_reference_and_sublayer/a_sphere.usda
#usda 1.0

def Sphere "a_sphere"
{
    double3 xformOp:translate = (0.0, 1.0, 0.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
}

Writing examples/usd_beginner_guide/reference_example_2_reference_and_sublayer/a_sphere.usda


In [37]:
%%file examples/usd_beginner_guide/reference_example_2_reference_and_sublayer/reference_and_sublayer_a_sphere.usda
#usda 1.0

(
    # I can't add this sublayer more that once.
    # I can't modify the sublayer here even I can't rename it. 
    # any modifications should be done in the original layer usd file.
    subLayers = [
        @./a_sphere.usda@
    ]
)

# But I can reference it as many times. 
# Note that my reference transform overrides the original layer transform.
def "sphere_02"(
    prepend references  = @./a_sphere.usda@</a_sphere>
)
{
    double3 xformOp:translate = (0.0, 0.0, 2.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 
def "sphere_03"(
    prepend references  = @./a_sphere.usda@</a_sphere>
)
{
    double3 xformOp:translate = (0.0, 0.0, 4.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 
def "sphere_04"(
    prepend references  = @./a_sphere.usda@</a_sphere>
)
{
    double3 xformOp:translate = (0.0, 0.0, 6.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 

Writing examples/usd_beginner_guide/reference_example_2_reference_and_sublayer/reference_and_sublayer_a_sphere.usda


**Result**:
![](../resources/getting_started_guide/reference_example_2_reference_and_sublayer.png)

#### **Payload**

A reference that can be loaded in a deferred way.

**Example 1: payload a layer**

In [39]:
%%file examples/usd_beginner_guide/payload_example/a_sphere.usda
#usda 1.0

def Sphere "a_sphere"
{}

Writing examples/usd_beginner_guide/payload_example/a_sphere.usda


In [40]:
%%file examples/usd_beginner_guide/payload_example/payload_a_sphere.usda
#usda 1.0

# referencing is done by choosing your layer followed by
# the desired prim to be referenced.
def "sphere_01"(
    prepend payload = @./a_sphere.usda@</a_sphere> 
)
{} 
# you can reference the same Prim as much as you want.
def "sphere_02"(
    prepend payload = @./a_sphere.usda@</a_sphere>
)
{
    double3 xformOp:translate = (0.0, 0.0, 2.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 
def "sphere_03"(
    prepend payload = @./a_sphere.usda@</a_sphere>
)
{
    double3 xformOp:translate = (0.0, 0.0, 4.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 
def "sphere_04"(
    prepend payload = @./a_sphere.usda@</a_sphere>
)
{
    double3 xformOp:translate = (0.0, 0.0, 6.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 

Writing examples/usd_beginner_guide/payload_example/payload_a_sphere.usda


**Result:**
![](../resources/getting_started_guide/payload_example.gif)

#### **Inherit**

A reference that used to point to other prims in the same layer.

**Example 1: Inherit a Prim**

In [44]:
%%file examples/usd_beginner_guide/inherit_example_1_inherit_a_prim/inherit_a_prim.usda
#usda 1.0

# Inherits only point to a prim that exist in same layer.
# I'll define a sphere to inherit later.
def Sphere "a_sphere" 
{
    double3 xformOp:translate = (0.0, 1.0, 0.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
}


# you can inherit the same Prim as much as you want.
def "sphere_02"(
    prepend inherits = </a_sphere>
)
{
    # I can override its properties! 
    color3f[] primvars:displayColor = [(0, 0, 1)] (
        interpolation = "constant"
    )
    double3 xformOp:translate = (0.0, 0.0, 2.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 
def "sphere_03"(
    prepend inherits = </a_sphere>
)
{
    double3 xformOp:translate = (0.0, 0.0, 4.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 
def "sphere_04"(
    prepend inherits = </a_sphere>
)
{
    double3 xformOp:translate = (0.0, 0.0, 6.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 

Writing examples/usd_beginner_guide/inherit_example_1_inherit_a_prim/inherit_a_prim.usda


**Result:**

![](../resources/getting_started_guide/inherit_example_1_inherit_a_prim.png)

**Example 2: Inherit a Prim from a referenced layer**

In [48]:
%%file examples/usd_beginner_guide/inherit_example_2_inherit_a_referenced_layer/a_sphere.usda
#usda 1.0

def Sphere "a_sphere"
{
    double3 xformOp:translate = (0.0, 1.0, 0.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
}

Overwriting examples/usd_beginner_guide/inherit_example_2_inherit_a_referenced_layer/a_sphere.usda


In [83]:
%%file examples/usd_beginner_guide/inherit_example_2_inherit_a_referenced_layer/inherit_a_referenced_layer.usda
#usda 1.0

# Inherits only point to a prim that exist in same layer.
# In order to inherit a prim from other layers, we need to reference it first.
def "a_sphere" (
    prepend references  = @./a_sphere.usda@</a_sphere>
)
{}


# you can inherit the same Prim as much as you want.
def "sphere_02"(
    prepend inherits = </a_sphere>
)
{
    # I can override its properties! 
    color3f[] primvars:displayColor = [(0, 0, 1)] (
        interpolation = "constant"
    )
    double3 xformOp:translate = (0.0, 0.0, 2.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 
def "sphere_03"(
    prepend inherits = </a_sphere>
)
{
    double3 xformOp:translate = (0.0, 0.0, 4.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 
def "sphere_04"(
    prepend inherits = </a_sphere>
)
{
    double3 xformOp:translate = (0.0, 0.0, 6.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 

Overwriting examples/usd_beginner_guide/inherit_example_2_inherit_a_referenced_layer/inherit_a_referenced_layer.usda


**Result:**
![](../resources/getting_started_guide/inherit_example_2_inherit_a_referenced_layer.png)

**Example 3: Inherit a Prim Class from a referenced layer**
This is how we mostly use inherits.

In [50]:
%%file examples/usd_beginner_guide/inherit_example_3_inherit_a_prim_class/a_sphere.usda
#usda 1.0

def Sphere "a_sphere"
{
    double3 xformOp:translate = (0.0, 1.0, 0.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
}

Writing examples/usd_beginner_guide/inherit_example_3_inherit_a_prim_class/a_sphere.usda


In [51]:
%%file examples/usd_beginner_guide/inherit_example_3_inherit_a_prim_class/inherit_a_sphere.usda
#usda 1.0

# Defining my prim class from a referenced layer.
class "_class_RedSpherePrimClass" (
    prepend references = @./a_sphere.usda@</a_sphere>
)
{
    # Here, I can override its properties for all inherits in this file
    color3f[] primvars:displayColor = [(1, 0, 0)] (
        interpolation = "constant"
    )
}


def "sphere_01"(
    prepend inherits = </_class_RedSpherePrimClass> 
)
{} 
# you can inherit the same Prim as much as you want.
def "sphere_02"(
    prepend inherits = </_class_RedSpherePrimClass>
)
{
    # Here, I can override its properties for this specific inherit
    color3f[] primvars:displayColor = [(0, 0, 1)] (
        interpolation = "constant"
    )
    double3 xformOp:translate = (0.0, 0.0, 2.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 
def "sphere_03"(
    prepend inherits = </_class_RedSpherePrimClass>
)
{
    double3 xformOp:translate = (0.0, 0.0, 4.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 
def "sphere_04"(
    prepend inherits = </_class_RedSpherePrimClass>
)
{
    double3 xformOp:translate = (0.0, 0.0, 6.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 

Writing examples/usd_beginner_guide/inherit_example_3_inherit_a_prim_class/inherit_a_sphere.usda


**Result:**
![](../resources/getting_started_guide/inherit_example_3_inherit_a_prim_class.png)

**Example 4 : Inherit and override prim class**

In [53]:
%%file examples/usd_beginner_guide/inherit_example_4_inherit_and_override_class/a_sphere.usda
#usda 1.0

def Sphere "a_sphere"
{
    double3 xformOp:translate = (0.0, 1.0, 0.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
}

Writing examples/usd_beginner_guide/inherit_example_4_inherit_and_override_class/a_sphere.usda


In [54]:
%%file examples/usd_beginner_guide/inherit_example_4_inherit_and_override_class/inherit_a_sphere.usda
#usda 1.0

# Defining my prim class from a referenced layer.
class "_class_RedSpherePrimClass" (
    prepend references = @./a_sphere.usda@</a_sphere>
)
{
    # Here, I can override its properties for all inherits in this file
    color3f[] primvars:displayColor = [(1, 0, 0)] (
        interpolation = "constant"
    )
}


def "sphere_01"(
    prepend inherits = </_class_RedSpherePrimClass> 
)
{} 
# you can inherit the same Prim as much as you want.
def "sphere_02"(
    prepend inherits = </_class_RedSpherePrimClass>
)
{
    # Here, I can override its properties for this specific inherit
    color3f[] primvars:displayColor = [(0, 0, 1)] (
        interpolation = "constant"
    )
    double3 xformOp:translate = (0.0, 0.0, 2.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 
def "sphere_03"(
    prepend inherits = </_class_RedSpherePrimClass>
)
{
    double3 xformOp:translate = (0.0, 0.0, 4.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 
def "sphere_04"(
    prepend inherits = </_class_RedSpherePrimClass>
)
{
    double3 xformOp:translate = (0.0, 0.0, 6.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
}

Writing examples/usd_beginner_guide/inherit_example_4_inherit_and_override_class/inherit_a_sphere.usda


In [55]:
%%file examples/usd_beginner_guide/inherit_example_4_inherit_and_override_class/turn_sphere_class_green.usda
#usda 1.0

# Defining my prim class from a referenced layer.
over "_class_RedSpherePrimClass" (
    prepend references = @./a_sphere.usda@</a_sphere>
)
{
    # Here, I can override its properties for all inherits in this file
    color3f[] primvars:displayColor = [(0, 1, 0)] (
        interpolation = "constant"
    )
}

Writing examples/usd_beginner_guide/inherit_example_4_inherit_and_override_class/turn_sphere_class_green.usda


In [56]:
%%file examples/usd_beginner_guide/inherit_example_4_inherit_and_override_class/sub_layer_inherit_and_modification.usda
#usda 1.0

(
    subLayers = [
        @./turn_sphere_class_green.usda@,
        @./inherit_a_sphere.usda@,
    ]
)

Writing examples/usd_beginner_guide/inherit_example_4_inherit_and_override_class/sub_layer_inherit_and_modification.usda


**Result:** Notice the layer stack
![](../resources/getting_started_guide/inherit_example_4_inherit_and_override_class.png)

**Example 5 : Inherit and override individual prims**

In [57]:
%%file examples/usd_beginner_guide/inherit_example_5_inherit_and_override_individual_prims/a_sphere.usda
#usda 1.0

def Sphere "a_sphere"
{
    double3 xformOp:translate = (0.0, 1.0, 0.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
}

Writing examples/usd_beginner_guide/inherit_example_5_inherit_and_override_individual_prims/a_sphere.usda


In [58]:
%%file examples/usd_beginner_guide/inherit_example_5_inherit_and_override_individual_prims/inherit_a_sphere.usda
#usda 1.0

# Defining my prim class from a referenced layer.
class "_class_RedSpherePrimClass" (
    prepend references = @./a_sphere.usda@</a_sphere>
)
{
    # Here, I can override its properties for all inherits in this file
    color3f[] primvars:displayColor = [(1, 0, 0)] (
        interpolation = "constant"
    )
}


def "sphere_01"(
    prepend inherits = </_class_RedSpherePrimClass> 
)
{} 
# you can inherit the same Prim as much as you want.
def "sphere_02"(
    prepend inherits = </_class_RedSpherePrimClass>
)
{
    # Here, I can override its properties for this specific inherit
    color3f[] primvars:displayColor = [(0, 0, 1)] (
        interpolation = "constant"
    )
    double3 xformOp:translate = (0.0, 0.0, 2.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 
def "sphere_03"(
    prepend inherits = </_class_RedSpherePrimClass>
)
{
    double3 xformOp:translate = (0.0, 0.0, 4.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 
def "sphere_04"(
    prepend inherits = </_class_RedSpherePrimClass>
)
{
    double3 xformOp:translate = (0.0, 0.0, 6.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 

Writing examples/usd_beginner_guide/inherit_example_5_inherit_and_override_individual_prims/inherit_a_sphere.usda


In [84]:
%%file examples/usd_beginner_guide/inherit_example_5_inherit_and_override_individual_prims/turn_spheres_green.usda
#usda 1.0


over "sphere_01"
{
    color3f[] primvars:displayColor = [(0, 1, 0)] (
        interpolation = "constant"
    )
}
over "sphere_02"
{
    color3f[] primvars:displayColor = [(0, 1, 0)] (
        interpolation = "constant"
    )
}
over "sphere_03"
{
    color3f[] primvars:displayColor = [(0, 1, 0)] (
        interpolation = "constant"
    )
}
over "sphere_04"
{
    color3f[] primvars:displayColor = [(0, 1, 0)] (
        interpolation = "constant"
    )
}

Overwriting examples/usd_beginner_guide/inherit_example_5_inherit_and_override_individual_prims/turn_spheres_green.usda


In [85]:
%%file examples/usd_beginner_guide/inherit_example_5_inherit_and_override_individual_prims/sub_layer_inherit_and_modification.usda
#usda 1.0

(
    subLayers = [
        @./turn_spheres_green.usda@,
        @./inherit_a_sphere.usda@,
    ]
)

Overwriting examples/usd_beginner_guide/inherit_example_5_inherit_and_override_individual_prims/sub_layer_inherit_and_modification.usda


**Result:**
![](../resources/getting_started_guide/inherit_example_5_inherit_and_override_individual_prims.png)

**Example 6: Inherit and override original prim**

Inherits works better with `class` but what happens if we used a normal `prim` ? .. let's find out.

In [61]:
%%file examples/usd_beginner_guide/inherit_example_6_inherit_and_override_original_prim/a_sphere.usda
#usda 1.0

def Sphere "a_sphere"
{
    double3 xformOp:translate = (0.0, 1.0, 0.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
}

Writing examples/usd_beginner_guide/inherit_example_6_inherit_and_override_original_prim/a_sphere.usda


In [62]:
%%file examples/usd_beginner_guide/inherit_example_6_inherit_and_override_original_prim/inherit_a_sphere.usda
#usda 1.0

def "a_sphere" (
    prepend references = @./a_sphere.usda@</a_sphere>
)
{
    # Here, I can override its properties for all inherits in this file
    color3f[] primvars:displayColor = [(1, 0, 0)] (
        interpolation = "constant"
    )
}


def "sphere_01"(
    prepend inherits = </a_sphere> 
)
{} 
# you can inherit the same Prim as much as you want.
def "sphere_02"(
    prepend inherits = </a_sphere>
)
{
    # Here, I can override its properties for this specific inherit
    color3f[] primvars:displayColor = [(0, 0, 1)] (
        interpolation = "constant"
    )
    double3 xformOp:translate = (0.0, 0.0, 2.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 
def "sphere_03"(
    prepend inherits = </a_sphere>
)
{
    double3 xformOp:translate = (0.0, 0.0, 4.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 
def "sphere_04"(
    prepend inherits = </a_sphere>
)
{
    double3 xformOp:translate = (0.0, 0.0, 6.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 

Writing examples/usd_beginner_guide/inherit_example_6_inherit_and_override_original_prim/inherit_a_sphere.usda


In [63]:
%%file examples/usd_beginner_guide/inherit_example_6_inherit_and_override_original_prim/reference_my_spheres.usda
#usda 1.0


def "sphere_01"(
    prepend references = @./inherit_a_sphere.usda@</sphere_01>
)
{} 

def "sphere_02"(
    prepend references = @./inherit_a_sphere.usda@</sphere_02>
)
{} 

def "sphere_03"(
    prepend references = @./inherit_a_sphere.usda@</sphere_03>
)
{}

def "sphere_04"(
    prepend references = @./inherit_a_sphere.usda@</sphere_04>
)
{} 

Writing examples/usd_beginner_guide/inherit_example_6_inherit_and_override_original_prim/reference_my_spheres.usda


In [64]:
%%file examples/usd_beginner_guide/inherit_example_6_inherit_and_override_original_prim/turn_original_sphere_green.usda
#usda 1.0

over "a_sphere"
{
    color3f[] primvars:displayColor = [(0, 1, 0)] (
        interpolation = "constant"
    )
}

Writing examples/usd_beginner_guide/inherit_example_6_inherit_and_override_original_prim/turn_original_sphere_green.usda


In [65]:
%%file examples/usd_beginner_guide/inherit_example_6_inherit_and_override_original_prim/sub_layer_inherit_and_modification.usda
#usda 1.0

(
    subLayers = [
        @./turn_original_sphere_green.usda@,
        @./reference_my_spheres.usda@,
    ]
)

Writing examples/usd_beginner_guide/inherit_example_6_inherit_and_override_original_prim/sub_layer_inherit_and_modification.usda


**Result:**
![](../resources/getting_started_guide/inherit_example_6_inherit_and_override_original_prim.png)

#### **Specialize**

They are similar to inherits. but, a little different!

we don't use `class` with **specialize**.

**Example 1: specialize and override original prim**

> Although, this is exactly the previous **Example 6: Inherit and override original prim** (we only replace `inherits` with `specializes`) . But, it works in a similar manner to **Example 4 : Inherit and override prim class** in Inherit.


In [67]:
%%file examples/usd_beginner_guide/specialize_example_1_specialize_and_override_original_prim/a_sphere.usda
#usda 1.0

def Sphere "a_sphere"
{
    double3 xformOp:translate = (0.0, 1.0, 0.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
}

Writing examples/usd_beginner_guide/specialize_example_1_specialize_and_override_original_prim/a_sphere.usda


In [74]:
%%file examples/usd_beginner_guide/specialize_example_1_specialize_and_override_original_prim/specialize_a_sphere.usda
#usda 1.0

def "a_sphere" (
    prepend references = @./a_sphere.usda@</a_sphere>
)
{
    # Here, I can override its properties for all inherits in this file
    color3f[] primvars:displayColor = [(1, 0, 0)] (
        interpolation = "constant"
    )
}


def "sphere_01"(
    prepend specializes = </a_sphere> 
)
{} 
# you can inherit the same Prim as much as you want.
def "sphere_02"(
    prepend specializes = </a_sphere>
)
{
    # Here, I can override its properties for this specific inherit
    color3f[] primvars:displayColor = [(0, 0, 1)] (
        interpolation = "constant"
    )
    double3 xformOp:translate = (0.0, 0.0, 2.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 
def "sphere_03"(
    prepend specializes = </a_sphere>
)
{
    double3 xformOp:translate = (0.0, 0.0, 4.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 
def "sphere_04"(
    prepend specializes = </a_sphere>
)
{
    double3 xformOp:translate = (0.0, 0.0, 6.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 

Overwriting examples/usd_beginner_guide/specialize_example_1_specialize_and_override_original_prim/specialize_a_sphere.usda


In [73]:
%%file examples/usd_beginner_guide/specialize_example_1_specialize_and_override_original_prim/reference_my_spheres.usda
#usda 1.0

# when using inherits, I don't have to worry about the class prim because it's not shown anyway.
# unlike specializers I need to reference my specialized Prims in a seperate layer.

def "sphere_01"(
    prepend references = @./specialize_a_sphere.usda@</sphere_01>
)
{} 

def "sphere_02"(
    prepend references = @./specialize_a_sphere.usda@</sphere_02>
)
{} 

def "sphere_03"(
    prepend references = @./specialize_a_sphere.usda@</sphere_03>
)
{}

def "sphere_04"(
    prepend references = @./specialize_a_sphere.usda@</sphere_04>
)
{} 

Overwriting examples/usd_beginner_guide/specialize_example_1_specialize_and_override_original_prim/reference_my_spheres.usda


In [70]:
%%file examples/usd_beginner_guide/specialize_example_1_specialize_and_override_original_prim/turn_original_sphere_green.usda
#usda 1.0

# Note that this Prim doesn't exist in the previous file 'reference_my_spheres.usda'
over "a_sphere"
{
    color3f[] primvars:displayColor = [(0, 1, 0)] (
        interpolation = "constant"
    )
}

Writing examples/usd_beginner_guide/specialize_example_1_specialize_and_override_original_prim/turn_original_sphere_green.usda


In [72]:
%%file examples/usd_beginner_guide/specialize_example_1_specialize_and_override_original_prim/sub_layer_specialize_and_modification.usda
#usda 1.0

(
    subLayers = [
        @./turn_original_sphere_green.usda@,
        @./reference_my_spheres.usda@,
    ]
)

Overwriting examples/usd_beginner_guide/specialize_example_1_specialize_and_override_original_prim/sub_layer_specialize_and_modification.usda


**Result:** Notice the Stack Layer
![](../resources/getting_started_guide/specialize_example_1_specialize_and_override_original_prim.png)

**Example 2: specialize and override specialized prims**

> This works in a similar manner to **Example 5 : Inherit and override individual prims** in Inherit.

In [75]:
%%file examples/usd_beginner_guide/specialize_example_2_specialize_and_override_specialized_prims/a_sphere.usda
#usda 1.0

def Sphere "a_sphere"
{
    double3 xformOp:translate = (0.0, 1.0, 0.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
}

Writing examples/usd_beginner_guide/specialize_example_2_specialize_and_override_specialized_prims/a_sphere.usda


In [76]:
%%file examples/usd_beginner_guide/specialize_example_2_specialize_and_override_specialized_prims/specialize_a_sphere.usda
#usda 1.0

def "a_sphere" (
    prepend references = @./a_sphere.usda@</a_sphere>
)
{
    # Here, I can override its properties for all inherits in this file
    color3f[] primvars:displayColor = [(1, 0, 0)] (
        interpolation = "constant"
    )
}


def "sphere_01"(
    prepend specializes = </a_sphere> 
)
{} 
# you can inherit the same Prim as much as you want.
def "sphere_02"(
    prepend specializes = </a_sphere>
)
{
    # Here, I can override its properties for this specific inherit
    color3f[] primvars:displayColor = [(0, 0, 1)] (
        interpolation = "constant"
    )
    double3 xformOp:translate = (0.0, 0.0, 2.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 
def "sphere_03"(
    prepend specializes = </a_sphere>
)
{
    double3 xformOp:translate = (0.0, 0.0, 4.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
} 
def "sphere_04"(
    prepend specializes = </a_sphere>
)
{
    double3 xformOp:translate = (0.0, 0.0, 6.0)
    uniform token[] xformOpOrder = ["xformOp:translate"]
}

Writing examples/usd_beginner_guide/specialize_example_2_specialize_and_override_specialized_prims/specialize_a_sphere.usda


In [77]:
%%file examples/usd_beginner_guide/specialize_example_2_specialize_and_override_specialized_prims/reference_my_spheres.usda
#usda 1.0

# when using inherits, I don't have to worry about the class prim because it's not shown anyway.
# unlike specializers I need to reference my specialized Prims in a seperate layer.

def "sphere_01"(
    prepend references = @./specialize_a_sphere.usda@</sphere_01>
)
{} 

def "sphere_02"(
    prepend references = @./specialize_a_sphere.usda@</sphere_02>
)
{} 

def "sphere_03"(
    prepend references = @./specialize_a_sphere.usda@</sphere_03>
)
{}

def "sphere_04"(
    prepend references = @./specialize_a_sphere.usda@</sphere_04>
)
{}

Writing examples/usd_beginner_guide/specialize_example_2_specialize_and_override_specialized_prims/reference_my_spheres.usda


In [79]:
%%file examples/usd_beginner_guide/specialize_example_2_specialize_and_override_specialized_prims/turn_spheres_green.usda
#usda 1.0


over "sphere_01"
{
    # Here, I can override its properties for all inherits in this file
    color3f[] primvars:displayColor = [(0, 1, 0)] (
        interpolation = "constant"
    )   
}
over "sphere_02"
{
    # Here, I can override its properties for all inherits in this file
    color3f[] primvars:displayColor = [(0, 1, 0)] (
        interpolation = "constant"
    )   
}
over "sphere_03"
{
    # Here, I can override its properties for all inherits in this file
    color3f[] primvars:displayColor = [(0, 1, 0)] (
        interpolation = "constant"
    )   
}
over "sphere_04"
{
    # Here, I can override its properties for all inherits in this file
    color3f[] primvars:displayColor = [(0, 1, 0)] (
        interpolation = "constant"
    )   
}

Writing examples/usd_beginner_guide/specialize_example_2_specialize_and_override_specialized_prims/turn_spheres_green.usda


In [78]:
%%file examples/usd_beginner_guide/specialize_example_2_specialize_and_override_specialized_prims/sub_layer_specialize_and_modification.usda
#usda 1.0

(
    subLayers = [
        @./turn_spheres_green.usda@,
        @./reference_my_spheres.usda@,
    ]
)

Writing examples/usd_beginner_guide/specialize_example_2_specialize_and_override_specialized_prims/sub_layer_specialize_and_modification.usda


**Result:** Notice the layer stack.
![](../resources/getting_started_guide/specialize_example_2_specialize_and_override_specialized_prims.png)

#### **Inherits and Specialize**

Recalling **Example 6: Inherit and override original prim** and **Example 1: specialize and override original prim**

What was so special about `"sphere_02"` . well, let's have a look at the **layer stack** and see when the prim is turned green.
    

| | **Why is it green?** | **Why is it blue?** |
| -- | -- | -- | 
| **Layer Stack** | ![](../resources/getting_started_guide/inherits_and_specialize_why_is_it_green.png) | ![](../resources/getting_started_guide/inherits_and_specialize_why_is_it_blue.png) |
| **Composition Inspection** | ![](../resources/getting_started_guide/inherits_and_specialize_why_is_it_green_2.png) | ![](../resources/getting_started_guide/inherits_and_specialize_why_is_it_blue_2.png) |

## **USD Composition Engine**

You can now infer why composition arcs can lead to confusion.

Learning about how composition engine works helps a lot in avoiding such a confusion.

> There's a great presentation notes here [Siggraph presentation notes | Pixar](https://openusd.org/release/dl_downloads.html) by Pixar.
>
> In [Siggraph2019 USD Composition | Pixar](https://openusd.org/files/Siggraph2019_USD%20Composition.pdf), there's a stunning recap of Composition Arcs and Strength Order.

### **Strength Ordering (LIVRPS)**

Each composition arc has a “strength” assigned to it.
The composition arc that is the “strongest” wins.
USD resolves composition arcs and their opinions in this exact order LIVRPS (Liver Peas).
- Local
- Inherits
- Variants
- References
- Payloads
- Specializes

> Image credits: [Strength Ordering (LIVRPS)](https://remedy-entertainment.github.io/USDBook/terminology/LIVRPS.html)
 
![](../resources/getting_started_guide/composition_engine.png)

### **LIVRPS Examples**

I have already talked about this topic indirectly in the previous sections, where there are many examples. 
You can actually go back and compare these examples to the previous image. 
But, why don't we return to this topic in this section with more direct examples.

> FYI, There are many magnificient LIVRPS examples in this course [USD and LOPs for Houdini Artists | FXPHD](https://www.fxphd.com/details/642/). 

#### **Layer Stack**
Revisiting `cube_asset` that was created in Houdini.
Notice the layer stack. It tells how different layers are stacked. 

This asset consists of 3 layers: 
- model : which describes my `cube` geometry (simple cube)
- lookdev : which describes my `cube` material (simple Houdini's principledshader + set material to my cube)
- animation : which describes my `cube` animation (simple transform animation)

As shown in the following gif: The layer stack in this asset is as follows:
`model` ➜ `lookdev` ➜ `animation` 

**Each layer overrides its predecessor layers.**

![](../resources/getting_started_guide/livrps_example_layer_stack.gif)


#### **Local Vs. Sublayer**
Applying local edits after sublayering. 
**Local always wins!**
| Edits | Result |
| -- | -- | 
| ![](../resources/getting_started_guide/livrps_example_1_local_vs_sublayer_1.png) | ![](../resources/getting_started_guide/livrps_example_1_local_vs_sublayer_2.png) |

#### **Local Vs. Reference**
Applying local edits after referencing. 
**Local always wins!**
| Edits | Result |
| -- | -- | 
| ![](../resources/getting_started_guide/livrps_example_2_local_vs_reference_1.png) | ![](../resources/getting_started_guide/livrps_example_2_local_vs_reference_2.png) |

#### **Composition Inspection**
Show compoisiton results step by step. 
This should help you when you can't figure out why a certain outcome occurred.

Inspection can be done through compoisiton tab in usdview.

Recalling this example: 
Inspection is done through checking **Layer Stack** and **Composition** tabs. Specially **Compoisiton** tab which tells the composition history. 

| | **Layer Stack** | **Composition Inspection** |
| -- | -- | -- | 
| **Why is it green?** | ![](../resources/getting_started_guide/inherits_and_specialize_why_is_it_green.png) | ![](../resources/getting_started_guide/inherits_and_specialize_why_is_it_green_2.png) |


#### **More Examples are in their way**